# ECG Arrhythmia Classification Demo

This notebook demonstrates the full pipeline:
- Download MIT-BIH Arrhythmia (PhysioNet)
- Preprocess (segment beats around R, 1000 samples, z-score per segment)
- Train CNN+LSTM with class weights and early stopping
- Evaluate (10-fold stratified): accuracy, sensitivity, specificity, PPV, confusion matrix
- Run inference on a record and visualize a few predictions

Requirements: TensorFlow 2.x, wfdb, numpy, scipy, pandas, scikit-learn, matplotlib, seaborn.

In [ ]:
# Setup: imports and paths
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ecg_arrhythmia.config import DATA_DIR, MODELS_DIR, LOGS_DIR, TENSORBOARD_DIR, CLASSES
from ecg_arrhythmia.data_download import download_mitbih
from ecg_arrhythmia.dataset import build_dataset
from ecg_arrhythmia.model import build_model, expand_channels
from ecg_arrhythmia.train_eval import run_kfold_training
from ecg_arrhythmia.inference import predict_record

print('DATA_DIR=', DATA_DIR)

In [ ]:
# Optional: download dataset (run once)
# download_mitbih(DATA_DIR)
print('If needed, dataset will be downloaded by training script as well.')

In [ ]:
# Build dataset (may take several minutes)
X, y, rec_ids = build_dataset(DATA_DIR)
Xc = expand_channels(X)
X.shape, Xc.shape, np.bincount(y, minlength=len(CLASSES))

In [ ]:
# Train one fold quickly for demo (set epochs small)
res = run_kfold_training(DATA_DIR, kfolds=2, epochs=3, batch_size=128, patience=2)
res

In [ ]:
# Inference on a record (requires a trained model path)
# Example assumes models/fold_1.keras exists from training
from ecg_arrhythmia.inference import CLASS_IDX_TO_NAME
record_path = DATA_DIR / '100'
model_path = Path.cwd() / 'models' / 'fold_1.keras'
if model_path.exists():
    out = predict_record(record_path, model_path)
    unique, counts = np.unique(out['y_pred'], return_counts=True)
    print({CLASS_IDX_TO_NAME.get(int(k), '?'): int(v) for k, v in zip(unique, counts)})
else:
    print('Train a model first to run inference (models/fold_1.keras not found).')